# Peer-Graded Assignment : Segmenting and Clustering Neighborhoods in Paris
- Build a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name in Toronto.
- Get the geographical coordinates of the neighborhoods in Toronto.
- Explore and cluster the neighborhoods in Toronto (replicate the same analysis we did to New York City data).
***
### 1. Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

!pip install BeautifulSoup4
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

     |████████████████████████████████| 112kB 3.0MB/s eta 0:00:01
     |████████████████████████████████| 122kB 10.9MB/s eta 0:00:01     |███████████████████████████████▏| 112kB 10.9MB/s eta 0:00:01
Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame

In [2]:
# send the GET request
data = requests.get('https://fr.wikipedia.org/wiki/Liste_des_quartiers_administratifs_de_Paris').text

In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [4]:
# create three lists to store table data
arrondissements = []
boroughList = []
neighborhoodList = []

**Using BeautifulSoup**

```python
# find the table
soup.find('table').find_all('tr')

# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
```

In [5]:
# append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        print(cells[0].text)
        #postalCodeList.append(cells[0].text)
        #boroughList.append(cells[1].text)
        #neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell

1er arrondissementdit « du Louvre »

2e
3e
4e
2e arrondissementdit « de la Bourse »

6e
7e
8e
3e arrondissementdit « du Temple »

10e
11e
12e
4e arrondissementdit « de l'Hôtel-de-Ville »

14e
15e
16e
5e arrondissementdit « du Panthéon »

18e
19e
20e
6e arrondissementdit « du Luxembourg »

22e
23e
24e
7e arrondissementdit « du Palais-Bourbon »

26e
27e
28e
8e arrondissementdit « de l'Élysée »

30e
31e
32e
9e arrondissementdit « de l'Opéra »

34e
35e
36e
10e arrondissementdit « de l'Entrepôt »

38e
39e
40e
11e arrondissementdit « de Popincourt »

42e
43e
44e
12e arrondissementdit « de Reuilly »

46e
47e
48e
13e arrondissementdit « des Gobelins »

50e
51e
52e
14e arrondissementdit « de l'Observatoire »

54e
55e
56e
15e arrondissementdit « de Vaugirard »

58e
59e
60e
16e arrondissementdit « de Passy »

62e
63e
64e
17e arrondissementdit « des Batignolles-Monceaux »

66e
67e
68e
18e arrondissementdit « des Buttes-Montmartre »

70e
71e
72e
19e arrondissementdit « des Buttes-Chaumont »

74e
75

In [6]:
# create a new DataFrame from the three lists
paris_df = pd.read_csv('quartiers paris.csv')

paris_df.head()

,Arrondissement,QuartierNb,QuartierNom,Population,Superficie,Density,Plan
0,1er arrondissement dit « du Louvre »,1er,Saint-Germain-l'Auxerrois,1 672,86.9,1 924,Paris 1er arrondissement - Quartiers.svg
1,1er arrondissement dit « du Louvre »,2e,Halles,8 984,41.2,21 806,Paris 1er arrondissement - Quartiers.svg
2,1er arrondissement dit « du Louvre »,3e,Palais-Royal,3 195,27.4,11 661,Paris 1er arrondissement - Quartiers.svg
3,1er arrondissement dit « du Louvre »,4e,Place-Vendôme,3 044,26.9,11 316,Paris 1er arrondissement - Quartiers.svg
4,2e arrondissement dit « de la Bourse »,5e,Gaillon,1 345,18.8,7 154,Paris 2e arrondissement - Quartiers.svg


In [7]:
paris_df['Arr'] = paris_df['Arrondissement'].str.extract('(\d{1,2})', expand=True)
paris_df.head()

,Arrondissement,QuartierNb,QuartierNom,Population,Superficie,Density,Plan,Arr
0,1er arrondissement dit « du Louvre »,1er,Saint-Germain-l'Auxerrois,1 672,86.9,1 924,Paris 1er arrondissement - Quartiers.svg,1
1,1er arrondissement dit « du Louvre »,2e,Halles,8 984,41.2,21 806,Paris 1er arrondissement - Quartiers.svg,1
2,1er arrondissement dit « du Louvre »,3e,Palais-Royal,3 195,27.4,11 661,Paris 1er arrondissement - Quartiers.svg,1
3,1er arrondissement dit « du Louvre »,4e,Place-Vendôme,3 044,26.9,11 316,Paris 1er arrondissement - Quartiers.svg,1
4,2e arrondissement dit « de la Bourse »,5e,Gaillon,1 345,18.8,7 154,Paris 2e arrondissement - Quartiers.svg,2


In [8]:
paris_df['CodePostal'] = '750' + paris_df['Arr'].apply(lambda x: '{0:0>2}'.format(x))
paris_df.head()

,Arrondissement,QuartierNb,QuartierNom,Population,Superficie,Density,Plan,Arr,CodePostal
0,1er arrondissement dit « du Louvre »,1er,Saint-Germain-l'Auxerrois,1 672,86.9,1 924,Paris 1er arrondissement - Quartiers.svg,1,75001
1,1er arrondissement dit « du Louvre »,2e,Halles,8 984,41.2,21 806,Paris 1er arrondissement - Quartiers.svg,1,75001
2,1er arrondissement dit « du Louvre »,3e,Palais-Royal,3 195,27.4,11 661,Paris 1er arrondissement - Quartiers.svg,1,75001
3,1er arrondissement dit « du Louvre »,4e,Place-Vendôme,3 044,26.9,11 316,Paris 1er arrondissement - Quartiers.svg,1,75001
4,2e arrondissement dit « de la Bourse »,5e,Gaillon,1 345,18.8,7 154,Paris 2e arrondissement - Quartiers.svg,2,75002


### 3. Drop cells with a borough that is "Not assigned"

In [ ]:
# drop cells with a borough that is Not assigned
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned\n"].reset_index(drop=True)
toronto_df_dropna.head()

### 4. Group neighborhoods in the same borough


In [ ]:
# group neighborhoods in the same borough
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

### 5. For Neighborhood="Not assigned", make the value the same as Borough

In [ ]:
# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

### 6. Check whether it is the same as required by the question

In [ ]:
# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    postcode += '\n'
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

### 7. Print the number of rows of the cleaned dataframe

In [ ]:
# print the number of rows of the cleaned dataframe
toronto_df_grouped.shape

### 8. Load the coordinates

In [9]:
latitudes =[]
longitudes =[]
for address, city in zip(paris_df['QuartierNom'], paris_df['CodePostal']):
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitudes.append(location.latitude)
    longitudes.append(location.longitude)
    #print('The geograpical coordinates of {} are {}, {}.'.format(address, location.latitude, location.longitude))

In [10]:
paris_df['Latitude'] = latitudes
paris_df['Longitude'] = longitudes
paris_df.head()

,Arrondissement,QuartierNb,QuartierNom,Population,Superficie,Density,Plan,Arr,CodePostal,Latitude,Longitude
0,1er arrondissement dit « du Louvre »,1er,Saint-Germain-l'Auxerrois,1 672,86.9,1 924,Paris 1er arrondissement - Quartiers.svg,1,75001,48.860211,2.336299
1,1er arrondissement dit « du Louvre »,2e,Halles,8 984,41.2,21 806,Paris 1er arrondissement - Quartiers.svg,1,75001,46.118907,4.957093
2,1er arrondissement dit « du Louvre »,3e,Palais-Royal,3 195,27.4,11 661,Paris 1er arrondissement - Quartiers.svg,1,75001,48.863585,2.336204
3,1er arrondissement dit « du Louvre »,4e,Place-Vendôme,3 044,26.9,11 316,Paris 1er arrondissement - Quartiers.svg,1,75001,48.867463,2.329428
4,2e arrondissement dit « de la Bourse »,5e,Gaillon,1 345,18.8,7 154,Paris 2e arrondissement - Quartiers.svg,2,75002,49.160560,1.333613


### 9. Merge two tables to get the coordinates

In [ ]:
toronto_df_grouped['PostalCode'] = toronto_df_grouped['PostalCode'].replace({'\n':''}, regex=True)
toronto_df_grouped.head()

In [ ]:
# merge two table on the column "PostalCode"
toronto_df_new = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()

### 10. Finally, check to make sure the coordinates are added as required by the question

In [ ]:
# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_new[toronto_df_new["PostalCode"]==postcode], ignore_index=True)
    
test_df

### 11. Use geopy library to get the latitude and longitude values of Paris

In [11]:
address = 'Paris'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris are 48.8566969, 2.3514616.


### 12. Create a map of Paris with neighborhoods superimposed on top

In [12]:
# create map of Toronto using latitude and longitude values
map_paris = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, arr, quar in zip(paris_df['Latitude'], paris_df['Longitude'], paris_df['Arrondissement'], paris_df['QuartierNom']):
    label = '{}, {}'.format(arr, quar)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_paris)  
    
map_paris 

### 13. Filter only boroughs that contain the word Toronto 

In [ ]:
# filter borough names that contain the word Toronto
borough_names = list(toronto_df_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

In [ ]:
# create a new DataFrame with only boroughs that contain the word Toronto
toronto_df_new = toronto_df_new[toronto_df_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_df_new.shape)
toronto_df_new.head()

In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto 

### 14. Use the Foursquare API to explore the neighborhoods

In [13]:
CLIENT_ID = '3RONWKOOH5SZF0VJYYNGT2A0UWYHI5NYLX1KBS3XPIKZM2EP' # your Foursquare ID
CLIENT_SECRET = 'SCLRW54FZTLVSHEC20JYNCD1H1K0ANJ4MTNURFT5TNY22X1B' # your Foursquare Secret
VERSION = '20201020'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3RONWKOOH5SZF0VJYYNGT2A0UWYHI5NYLX1KBS3XPIKZM2EP
CLIENT_SECRET:SCLRW54FZTLVSHEC20JYNCD1H1K0ANJ4MTNURFT5TNY22X1B


**Now, let's get the top 100 venues that are within a radius of 1000 meters.**

In [ ]:
radius = 1000
LIMIT = 100

venues = []

for lat, lng, arr, quar, post in zip(paris_df['Latitude'], paris_df['Longitude'], paris_df['Arrondissement'], paris_df['QuartierNom'], paris_df['CodePostal']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lng,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            arr,
            quar,
            lat, 
            lng, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [ ]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['CodePostal', 'Arrondissement', 'Quartier', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

In [14]:
#venues_df.to_csv('venues.csv')
venues_df = pd.read_csv('venues.csv')

In [16]:
print(venues_df.shape)
venues_df.head()

(5491, 10)


,Unnamed: 0,CodePostal,Arrondissement,Quartier,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,0,75001,1er arrondissement dit « du Louvre »,Saint-Germain-l'Auxerrois,48.860211,2.336299,Musée du Louvre,48.860847,2.336440,Art Museum
1,1,75001,1er arrondissement dit « du Louvre »,Saint-Germain-l'Auxerrois,48.860211,2.336299,Cour Carrée du Louvre,48.860360,2.338543,Pedestrian Plaza
2,2,75001,1er arrondissement dit « du Louvre »,Saint-Germain-l'Auxerrois,48.860211,2.336299,La Vénus de Milo (Vénus de Milo),48.859943,2.337234,Exhibit
3,3,75001,1er arrondissement dit « du Louvre »,Saint-Germain-l'Auxerrois,48.860211,2.336299,Cour Napoléon,48.861172,2.335088,Plaza
4,4,75001,1er arrondissement dit « du Louvre »,Saint-Germain-l'Auxerrois,48.860211,2.336299,Pont des Arts,48.858565,2.337635,Bridge


**Let's check how many venues were returned for each PostalCode**

In [17]:
venues_df.groupby(["CodePostal", "Arrondissement", "Quartier"]).count()

Unnamed: 0  \
CodePostal Arrondissement                                     Quartier                                 
75001      1er arrondissement dit « du Louvre »                Halles                              7   
                                                               Palais-Royal                      100   
                                                               Place-Vendôme                     100   
                                                               Saint-Germain-l'Auxerrois         100   
75002      2e arrondissement dit « de la Bourse »              Bonne-Nouvelle                    100   
                                                               Gaillon                             4   
                                                               Mail                              100   
                                                               Vivienne                          100   
75003      3e arrondissement dit « du Temple »                 Archives                          100   
                                                               Arts-et-Métiers                   100   
                                                               Enfants-Rouges                    100   
                                                               Sainte-Avoye                      100   
75004      4e arrondissement dit « de l'Hôtel-de-Ville »       Arsenal                           100   
                                                               Notre-Dame                         28   
                                                               Saint-Merri                       100   
75005      5e arrondissement dit « du Panthéon »               Jardin-des-Plantes                100   
                                                               Saint-Victor                        1   
                                                               Sorbonne                          100   
                                                               Val-de-Grâce                      100   
75006      6e arrondissement dit « du Luxembourg »             Monnaie                             2   
                                                               Notre-Dame-des-Champs             100   
                                                               Odéon                             100   
75007      7e arrondissement dit « du Palais-Bourbon »         Invalides                         100   
                                                               Saint-Thomas-d'Aquin              100   
                                                               École-Militaire                   100   
75008      8e arrondissement dit « de l'Élysée »               Champs-Élysées                    100   
                                                               Europe                              1   
                                                               Faubourg-du-Roule                 100   
                                                               Madeleine                          11   
75009      9e arrondissement dit « de l'Opéra »                Chaussée-d'Antin                  100   
                                                               Faubourg-Montmartre               100   
                                                               Rochechouart                        8   
75010      10e arrondissement dit « de l'Entrepôt »            Hôpital-Saint-Louis               100   
                                                               Porte-Saint-Denis                 100   
                                                               Porte-Saint-Martin                100   
                                                               Saint-Vincent-de-Paul               3   
75011      11e arrondissement dit « de Popincourt »            Folie-Méricourt                   100   
                                  

**Let's find out how many unique categories can be curated from all the returned venues**

In [18]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 311 uniques categories.


In [19]:
venues_df['VenueCategory'].unique()[:50]

array(['Art Museum', 'Pedestrian Plaza', 'Exhibit', 'Plaza', 'Bridge',
       'Historic Site', 'Theater', 'Cosmetics Shop', 'Church',
       'Italian Restaurant', 'Coffee Shop', 'Bar', 'Garden', 'Museum',
       'Hotel', 'Café', 'Sandwich Place', 'Shoe Store',
       'Chinese Restaurant', 'Tea Room', 'Udon Restaurant',
       'Furniture / Home Store', 'Spa', 'Wine Bar', 'Cheese Shop',
       'Restaurant', 'Park', 'Fountain', 'French Restaurant',
       'Cocktail Bar', 'Japanese Restaurant', 'Pastry Shop',
       'Ramen Restaurant', 'Art Gallery', 'Bakery', 'Bistro',
       'Korean Restaurant', 'Tapas Restaurant', 'Hotel Bar', 'Bookstore',
       'Wine Shop', 'Israeli Restaurant', 'Modern European Restaurant',
       'Shopping Mall', 'Electronics Store', 'Pharmacy',
       'Construction & Landscaping', 'Photography Studio', 'Supermarket',
       'Rental Car Location'], dtype=object)

In [24]:
paris_df[~paris_df['QuartierNom'].isin(venues_df['Quartier'])]

,Arrondissement,QuartierNb,QuartierNom,Population,Superficie,Density,Plan,Arr,CodePostal,Latitude,Longitude
13,4e arrondissement dit « de l'Hôtel-de-Ville »,14e,Saint-Gervais,10 587,42.2,25 088,Paris 4e arrondissement - Quartiers.svg,4,75004,45.202236,5.482023
23,6e arrondissement dit « du Luxembourg »,24e,Saint-Germain-des-Prés,5 154,28.2,18 277,Paris 6e arrondissement - Quartiers.svg,6,75006,45.341950,0.991289
27,7e arrondissement dit « du Palais-Bourbon »,28e,Gros-Caillou,25 156,138.2,18 203,Paris 7e arrondissement - Quartiers.svg,7,75007,50.253151,3.094070
32,9e arrondissement dit « de l'Opéra »,33e,Saint-Georges,20 850,71.7,29 079,Paris 9e arrondissement - Quartiers.svg,9,75009,50.358395,2.087431
42,11e arrondissement dit « de Popincourt »,43e,Roquette,47 520,117.2,40 546,Paris 11e arrondissement - Quartiers.svg,11,75011,49.251781,1.345226
56,15e arrondissement dit « de Vaugirard »,57e,Saint-Lambert,82 032,283.1,29 330,Paris 15e arrondissement - Quartiers.svg,15,75015,48.937427,-0.548238
71,18e arrondissement dit « des Buttes-Montmartre »,72e,Chapelle,24 037,134.8,17 832,Paris 18e arrondissement - Quartiers.svg,18,75018,49.742222,5.015556
72,19e arrondissement dit « des Buttes-Chaumont »,73e,Villette,53 650,128.6,41 718,Paris 19e arrondissement - Quartiers.svg,19,75019,48.911881,-0.543368


### 15. Analyze Each Area

In [25]:
# one hot encoding
paris_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

In [26]:
paris_onehot.shape

(5491, 311)

In [27]:
paris_onehot.head()

,Accessories Store,African Restaurant,Alsatian Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Auvergne Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basque Restaurant,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Quad,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Drive-in Theater,Eastern European Restaurant,Electronics Store,Emilia Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health Food Store,Heliport,Historic Site,History Museum,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jiangxi Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Lyonese Bouchon,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Photography Studio,Piadineria,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Portuguese Restaurant,Provençal Restaurant,Pub,Radio Station,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Club,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Savoyard Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shaanxi Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Southern / Soul Food Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,S

In [28]:
# add postal, borough and neighborhood column back to dataframe
paris_onehot['CodePostal'] = venues_df['CodePostal'] 
paris_onehot['Arrondissement'] = venues_df['Arrondissement'] 
paris_onehot['Quartier'] = venues_df['Quartier'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(paris_onehot.columns[-3:]) + list(paris_onehot.columns[:-3])
paris_onehot = paris_onehot[fixed_columns]

print(paris_onehot.shape)
paris_onehot.head()

(5491, 314)


,CodePostal,Arrondissement,Quartier,Accessories Store,African Restaurant,Alsatian Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Auvergne Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basque Restaurant,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Quad,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Drive-in Theater,Eastern European Restaurant,Electronics Store,Emilia Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health Food Store,Heliport,Historic Site,History Museum,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jiangxi Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Lyonese Bouchon,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Photography Studio,Piadineria,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Portuguese Restaurant,Provençal Restaurant,Pub,Radio Station,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Club,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Savoyard Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shaanxi Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Southern / Soul Food Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Spiritual Cent

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [29]:
paris_grouped = paris_onehot.groupby(["CodePostal", "Arrondissement", "Quartier"]).mean().reset_index()

print(paris_grouped.shape)
paris_grouped.head()

(72, 314)


,CodePostal,Arrondissement,Quartier,Accessories Store,African Restaurant,Alsatian Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Auvergne Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basque Restaurant,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Quad,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Drive-in Theater,Eastern European Restaurant,Electronics Store,Emilia Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health Food Store,Heliport,Historic Site,History Museum,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jiangxi Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Lyonese Bouchon,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Photography Studio,Piadineria,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Portuguese Restaurant,Provençal Restaurant,Pub,Radio Station,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Club,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Savoyard Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shaanxi Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Southern / Soul Food Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Spiritual Cent

Now let's create the new dataframe and display the top 10 venues for each PostalCode.

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['CodePostal', 'Arrondissement', 'Quartier']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

In [31]:
freqColumns

['1st Most Common Venue',
 '2nd Most Common Venue',
 '3rd Most Common Venue',
 '4th Most Common Venue',
 '5th Most Common Venue',
 '6th Most Common Venue',
 '7th Most Common Venue',
 '8th Most Common Venue',
 '9th Most Common Venue',
 '10th Most Common Venue']

In [32]:
# create a new dataframe
quartiers_venues_sorted = pd.DataFrame(columns=columns)
quartiers_venues_sorted['CodePostal'] = paris_grouped['CodePostal']
quartiers_venues_sorted['Arrondissement'] = paris_grouped['Arrondissement']
quartiers_venues_sorted['Quartier'] = paris_grouped['Quartier']

In [33]:
quartiers_venues_sorted.head()

,CodePostal,Arrondissement,Quartier,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,75001,1er arrondissement dit « du Louvre »,Halles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,75001,1er arrondissement dit « du Louvre »,Palais-Royal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,75001,1er arrondissement dit « du Louvre »,Place-Vendôme,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,75001,1er arrondissement dit « du Louvre »,Saint-Germain-l'Auxerrois,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,75002,2e arrondissement dit « de la Bourse »,Bonne-Nouvelle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
for ind in np.arange(paris_grouped.shape[0]):
    row_categories = paris_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    quartiers_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(quartiers_venues_sorted.shape)
quartiers_venues_sorted.head()

(72, 13)


,CodePostal,Arrondissement,Quartier,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,75001,1er arrondissement dit « du Louvre »,Halles,Shopping Mall,Electronics Store,Rental Car Location,Construction & Landscaping,Pharmacy,Supermarket,Photography Studio,Zoo Exhibit,Fast Food Restaurant,Exhibit
1,75001,1er arrondissement dit « du Louvre »,Palais-Royal,Japanese Restaurant,Plaza,French Restaurant,Hotel,Italian Restaurant,Bookstore,Café,Art Museum,Historic Site,Garden
2,75001,1er arrondissement dit « du Louvre »,Place-Vendôme,Hotel,Plaza,Boutique,Art Museum,Women's Store,Bookstore,Historic Site,Coffee Shop,Tea Room,Garden
3,75001,1er arrondissement dit « du Louvre »,Saint-Germain-l'Auxerrois,French Restaurant,Hotel,Plaza,Italian Restaurant,Art Museum,Historic Site,Japanese Restaurant,Exhibit,Coffee Shop,Restaurant
4,75002,2e arrondissement dit « de la Bourse »,Bonne-Nouvelle,Hotel,Cocktail Bar,Wine Bar,French Restaurant,Bar,Pizza Place,Pedestrian Plaza,Restaurant,Thai Restaurant,Bakery


### 16. Cluster Areas
Run k-means to cluster the Toronto areas into 5 clusters.

In [35]:
print(paris_grouped.shape)
paris_grouped.head()

(72, 314)


,CodePostal,Arrondissement,Quartier,Accessories Store,African Restaurant,Alsatian Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Auvergne Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basque Restaurant,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Quad,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Drive-in Theater,Eastern European Restaurant,Electronics Store,Emilia Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health Food Store,Heliport,Historic Site,History Museum,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jiangxi Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Lyonese Bouchon,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Photography Studio,Piadineria,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Portuguese Restaurant,Provençal Restaurant,Pub,Radio Station,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Club,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Savoyard Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shaanxi Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Southern / Soul Food Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Spiritual Cent

In [36]:
# set number of clusters
kclusters = 5

paris_grouped_clustering = paris_grouped.drop(["CodePostal", "Arrondissement", "Quartier"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(paris_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [37]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
paris_clusters = quartiers_venues_sorted.copy()
# add clustering labels
paris_clusters["Cluster Labels"] = kmeans.labels_
print(paris_clusters.shape)
paris_clusters.head()

(72, 14)


,CodePostal,Arrondissement,Quartier,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,75001,1er arrondissement dit « du Louvre »,Halles,Shopping Mall,Electronics Store,Rental Car Location,Construction & Landscaping,Pharmacy,Supermarket,Photography Studio,Zoo Exhibit,Fast Food Restaurant,Exhibit,0
1,75001,1er arrondissement dit « du Louvre »,Palais-Royal,Japanese Restaurant,Plaza,French Restaurant,Hotel,Italian Restaurant,Bookstore,Café,Art Museum,Historic Site,Garden,0
2,75001,1er arrondissement dit « du Louvre »,Place-Vendôme,Hotel,Plaza,Boutique,Art Museum,Women's Store,Bookstore,Historic Site,Coffee Shop,Tea Room,Garden,0
3,75001,1er arrondissement dit « du Louvre »,Saint-Germain-l'Auxerrois,French Restaurant,Hotel,Plaza,Italian Restaurant,Art Museum,Historic Site,Japanese Restaurant,Exhibit,Coffee Shop,Restaurant,0
4,75002,2e arrondissement dit « de la Bourse »,Bonne-Nouvelle,Hotel,Cocktail Bar,Wine Bar,French Restaurant,Bar,Pizza Place,Pedestrian Plaza,Restaurant,Thai Restaurant,Bakery,0


In [38]:
print(paris_df.shape)
paris_df.head()

(80, 11)


,Arrondissement,QuartierNb,QuartierNom,Population,Superficie,Density,Plan,Arr,CodePostal,Latitude,Longitude
0,1er arrondissement dit « du Louvre »,1er,Saint-Germain-l'Auxerrois,1 672,86.9,1 924,Paris 1er arrondissement - Quartiers.svg,1,75001,48.860211,2.336299
1,1er arrondissement dit « du Louvre »,2e,Halles,8 984,41.2,21 806,Paris 1er arrondissement - Quartiers.svg,1,75001,46.118907,4.957093
2,1er arrondissement dit « du Louvre »,3e,Palais-Royal,3 195,27.4,11 661,Paris 1er arrondissement - Quartiers.svg,1,75001,48.863585,2.336204
3,1er arrondissement dit « du Louvre »,4e,Place-Vendôme,3 044,26.9,11 316,Paris 1er arrondissement - Quartiers.svg,1,75001,48.867463,2.329428
4,2e arrondissement dit « de la Bourse »,5e,Gaillon,1 345,18.8,7 154,Paris 2e arrondissement - Quartiers.svg,2,75002,49.160560,1.333613


In [47]:
df = pd.merge(paris_clusters, paris_df[['QuartierNom', 'Latitude', 'Longitude']], how='left',left_on=['Quartier'],right_on=['QuartierNom'])
print(df.shape)
df.head()

(72, 17)


,CodePostal,Arrondissement,Quartier,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,QuartierNom,Latitude,Longitude
0,75001,1er arrondissement dit « du Louvre »,Halles,Shopping Mall,Electronics Store,Rental Car Location,Construction & Landscaping,Pharmacy,Supermarket,Photography Studio,Zoo Exhibit,Fast Food Restaurant,Exhibit,0,Halles,46.118907,4.957093
1,75001,1er arrondissement dit « du Louvre »,Palais-Royal,Japanese Restaurant,Plaza,French Restaurant,Hotel,Italian Restaurant,Bookstore,Café,Art Museum,Historic Site,Garden,0,Palais-Royal,48.863585,2.336204
2,75001,1er arrondissement dit « du Louvre »,Place-Vendôme,Hotel,Plaza,Boutique,Art Museum,Women's Store,Bookstore,Historic Site,Coffee Shop,Tea Room,Garden,0,Place-Vendôme,48.867463,2.329428
3,75001,1er arrondissement dit « du Louvre »,Saint-Germain-l'Auxerrois,French Restaurant,Hotel,Plaza,Italian Restaurant,Art Museum,Historic Site,Japanese Restaurant,Exhibit,Coffee Shop,Restaurant,0,Saint-Germain-l'Auxerrois,48.860211,2.336299
4,75002,2e arrondissement dit « de la Bourse »,Bonne-Nouvelle,Hotel,Cocktail Bar,Wine Bar,French Restaurant,Bar,Pizza Place,Pedestrian Plaza,Restaurant,Thai Restaurant,Bakery,0,Bonne-Nouvelle,48.870623,2.348750


In [85]:
df = pd.merge(paris_df[['QuartierNom', 'Latitude', 'Longitude']], paris_clusters, how='left',right_on=['Quartier'],left_on=['QuartierNom'])
print(df.shape)
df=df.drop('Quartier',1)
df["Cluster Labels"]=df["Cluster Labels"].fillna(-1)
df["Cluster Labels"]=df["Cluster Labels"].astype(int)
df["Cluster Labels"] = df["Cluster Labels"]+1
df.head()

(80, 17)


,QuartierNom,Latitude,Longitude,CodePostal,Arrondissement,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Saint-Germain-l'Auxerrois,48.860211,2.336299,75001.0,1er arrondissement dit « du Louvre »,French Restaurant,Hotel,Plaza,Italian Restaurant,Art Museum,Historic Site,Japanese Restaurant,Exhibit,Coffee Shop,Restaurant,1
1,Halles,46.118907,4.957093,75001.0,1er arrondissement dit « du Louvre »,Shopping Mall,Electronics Store,Rental Car Location,Construction & Landscaping,Pharmacy,Supermarket,Photography Studio,Zoo Exhibit,Fast Food Restaurant,Exhibit,1
2,Palais-Royal,48.863585,2.336204,75001.0,1er arrondissement dit « du Louvre »,Japanese Restaurant,Plaza,French Restaurant,Hotel,Italian Restaurant,Bookstore,Café,Art Museum,Historic Site,Garden,1
3,Place-Vendôme,48.867463,2.329428,75001.0,1er arrondissement dit « du Louvre »,Hotel,Plaza,Boutique,Art Museum,Women's Store,Bookstore,Historic Site,Coffee Shop,Tea Room,Garden,1
4,Gaillon,49.160560,1.333613,75002.0,2e arrondissement dit « de la Bourse »,Auto Garage,Rental Car Location,Construction & Landscaping,Supermarket,Zoo Exhibit,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,1


In [86]:
df.groupby(["Cluster Labels", "QuartierNom"]).count()

Latitude  Longitude  CodePostal  \
Cluster Labels QuartierNom                                                   
0               Chapelle                          1          1           0   
                Gros-Caillou                      1          1           0   
                Roquette                          1          1           0   
                Saint-Georges                     1          1           0   
                Saint-Germain-des-Prés            1          1           0   
                Saint-Gervais                     1          1           0   
                Saint-Lambert                     1          1           0   
                Villette                          1          1           0   
1               Amérique                          1          1           1   
                Archives                          1          1           1   
                Arsenal                           1          1           1   
                Arts-et-Métiers                   1          1           1   
                Batignolles                       1          1           1   
                Bel-Air                           1          1           1   
                Belleville                        1          1           1   
                Bercy                             1          1           1   
                Bonne-Nouvelle                    1          1           1   
                Chaillot                          1          1           1   
                Champs-Élysées                    1          1           1   
                Charonne                          1          1           1   
                Chaussée-d'Antin                  1          1           1   
                Clignancourt                      1          1           1   
                Combat                            1          1           1   
                Croulebarbe                       1          1           1   
                Enfants-Rouges                    1          1           1   
                Faubourg-Montmartre               1          1           1   
                Faubourg-du-Roule                 1          1           1   
                Folie-Méricourt                   1          1           1   
                Gaillon                           1          1           1   
                Gare                              1          1           1   
                Goutte-d'Or                       1          1           1   
                Grandes-Carrières                 1          1           1   
                Grenelle                          1          1           1   
                Halles                            1          1           1   
                Hôpital-Saint-Louis               1          1           1   
                Invalides                         1          1           1   
                Jardin-des-Plantes                1          1           1   
                Javel                             1          1           1   
                Madeleine                         1          1           1   
                Mail                              1          1           1   
                Maison-Blanche                    1          1           1   
                Monnaie                           1          1           1   
                Montparnasse                      1          1           1   
                Muette                            1          1           1   
                Necker                            1          1           1   
                Notre-Dame                        1          1           1   
                Notre-Dame-des-Champs             1          1           1   
                Odéon                             1          1           1   
                Palais-Royal                      1          1           1   
                Parc-de-Montsouris                1          1   

**Finally, let's visualize the resulting clusters**

In [90]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

kclusters=5
kclusters=6
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, quar, arr, cluster in zip(df['Latitude'], df['Longitude'], df['QuartierNom'], df['Arrondissement'], df['Cluster Labels']):
    label = folium.Popup('{} ({}) - Cluster {}'.format(quar, arr, cluster), parse_html=True)      
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [88]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

kclusters=5
kclusters=6
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, quar, arr, cluster in zip(df['Latitude'], df['Longitude'], df['QuartierNom'], df['Arrondissement'], df['Cluster Labels']):
    label = folium.Popup('{} ({}) - Cluster {}'.format(quar, arr, cluster), parse_html=True)
    if cluster == 0:
        print(cluster)
        print(label)
        print(rainbow[cluster-1])
        
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster+1],
            fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

0
#ff0000
0
#ff0000
0
#ff0000
0
#ff0000
0
#ff0000
0
#ff0000
0
#ff0000
0
#ff0000


### 17. Examine Clusters

#### Cluster 1

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 2

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 3

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 4

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Cluster 5

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Observations:
Most of the neighborhoods fall into Cluster 1 which are mostly business areas with cafe, restaurants, supermarkets etc. Cluster 2 is just a garden, Cluster 3 are playground and park, Cluster 4 park and swim school, and lastly Cluster 5 park and trail.